# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Ilustração-da-convolução-como-uma-média-ponderada-de-translações-da-imagem-de-entrada" data-toc-modified-id="Ilustração-da-convolução-como-uma-média-ponderada-de-translações-da-imagem-de-entrada-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Ilustração da convolução como uma média ponderada de translações da imagem de entrada</a></div><div class="lev2 toc-item"><a href="#Equação" data-toc-modified-id="Equação-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Equação</a></div><div class="lev2 toc-item"><a href="#Ilustração-da-convolução-1D" data-toc-modified-id="Ilustração-da-convolução-1D-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Ilustração da convolução 1D</a></div><div class="lev2 toc-item"><a href="#Ilustração-da-convolução-2D" data-toc-modified-id="Ilustração-da-convolução-2D-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Ilustração da convolução 2D</a></div><div class="lev2 toc-item"><a href="#Veja-também" data-toc-modified-id="Veja-também-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Veja também</a></div><div class="lev2 toc-item"><a href="#Páginas-Interessantes" data-toc-modified-id="Páginas-Interessantes-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Páginas Interessantes</a></div>

# Ilustração da convolução como uma média ponderada de translações da imagem de entrada

O objetivo desta demonstração é ilustrar a implementação da convolução discreta linear utilizando
o algoritmo da média ponderada das translações da imagem. Esta implementação é apropriada para o
NumPy pois as translações são implementadas na forma de fatiamento

In [ ]:
import numpy as np
import ia636 as ia
def iaconvdemo(f, h):
    f = f.astype(float)
    h = h.astype(float)

    # If the arrays are 1D, convert them to 2D for generality purposes
    if len(f.shape) == 1: f = f[np.newaxis,:]
    if len(h.shape) == 1: h = h[np.newaxis,:]
    #if f.size < h.size:  f, h = h, f # Always iterate over the smaller image
    g = np.zeros(np.array(f.shape) + np.array(h.shape) - 1) # Allocating memory to store the convolution result


    adshow(ia.ianshow(h), 'h')
    adshow(ia.ianshow(f), 'f')
    adshow(ia.ianshow(g), 'g = np.zeros(np.array(f.shape) + np.array(h.shape) - 1)')


    mask_h = zeros(h.shape,dtype = bool)
    mask_g = zeros(g.shape,dtype = bool)


    for i in xrange(h.shape[0]):
       for j in xrange(h.shape[1]):
          g[i:i+f.shape[0], j:j+f.shape[1]] += h[i,j] * f
          mask_h[i,j] = True
          mask_g[i:i+f.shape[0], j:j+f.shape[1]] = True
          adshow(ia.ianshow(h,mask_h), 'h[%d.%d]' %(i,j))
          adshow(ia.ianshow(h[i,j] * f), 'h[%d.%d]*f' %(i,j))
          adshow(ia.ianshow(g,mask_g), 'g[%d:%d, %d:%d] += h[%d,%d] * f' %(i,i+f.shape[0],j,j+f.shape[1],i,j))
          mask_h[i,j] = False
          mask_g[i:i+f.shape[0], j:j+f.shape[1]] = False

    return g

Na convolução linear, ambas as imagens são infinitas com valores zeros fora da área da matriz. As dimensões da imagem resultante
será a soma das dimensões de cada imagem menos 1.

## Equação

$$ \begin{matrix}
    (f \ast h)(x,y) &=&  \sum_{i=0}^{N-1} \sum_{j=0}^{M-1} f_{e}(x-i,y-j) h_{e}(i, j) \\
    f_{e}(x,y) &=& \left\{ \begin{array}{llcl} f(x,y), & 0 \leq x \leq A-1 & and & 0 \leq y \leq B-1  \\
                                                          0, & A \leq x \leq N-1 & or & B \leq y \leq M-1 \end{array}\right.\\
    h_{e}(x,y) &=& \left\{ \begin{array}{llcl} h(x,y), & 0 \leq x \leq C-1 & and & 0 \leq y \leq D-1  \\
                                                          0, & C \leq x \leq N-1 & or & D \leq y \leq M-1 \end{array}\right.\\
    N & \geq & A + C - 1 \\
    M & \geq & B + D - 1
   \end{matrix} $$

## Ilustração da convolução 1D

Esta primeira ilustração, temos uma imagem unidimensional com 6 elementos e o núcleo da convolução com 3 elementos. O resultado
da convolução terá 8 elementos. Acompanhe o passo a passo da execução do algoritmo. O algoritmo original é o `ia636:iaconv` que foi
trazido para esta página e incorporado visualizações ilustrativas durante a sua execução.

In [ ]:
f = array([0,1,2,3,4,5])
h = array([-1,0,1])

print('f =\n', f)
print('\nh = \n', h)
print('\n f*h = \n',iaconvdemo(f, h))

## Ilustração da convolução 2D

Temos o caso de imagem bidimensional sendo processada por uma máscara Sobel de cálculo de gradiente horizontal. Acompanhe o passo-a-passo:

In [ ]:
f = array([[1,1,1,1,1],
          [2,2,2,2,2],
          [3,3,3,3,3],
          [4,4,4,4,4],
          [5,5,5,5,5]])
h = array([[-1,0,1],
          [-2,0,2],
          [-1,0,1]])

print('f =\n', f)
print('\nh = \n', h)
print('\n f*h = \n',iaconvdemo(f, h))

## Veja também

- [ia898:conv](../src/conv.ipynb)

## Páginas Interessantes

- [Wikipedia - Convolution](http://en.wikipedia.org/wiki/Convolution)
